## authenticate with polygon api

In [53]:
import requests
import datetime

###############################
# PASTE API KEYS/SECRETS HERE #
###############################

polygon_api_key = "hPF3TAsr  REDACTED  6weY1ISuWa"



polygon_url = "https://api.polygon.io"

## find a stock

In [28]:
search_term = "tesla"

url = polygon_url + "/v2/reference/tickers"
params = {
    "sort": "ticker",
    "locale": "us",
    "search": search_term,
    "market": "stocks",
    "perpage": 10,
    "page": 1,
    "apiKey": polygon_api_key
}

r = requests.get(url, params=params)
if r.status_code == 200:
    tickers = r.json()["tickers"]
    print("tickers related to search query '%s':" % search_term)
    for t in tickers:
        print("  ticker:", t["ticker"])
        print("  name:", t["name"])
        print()
else:
    print("error: couldn't reach polygon api")

tickers related to search query 'tesla':
  ticker: TSLA
  name: Tesla, Inc. Common Stock

  ticker: TXLZF
  name: Tesla Exploratin Services Inc. Ordinary Shares (Canada)



## find a crypto

In [37]:
search_term = "dogecoin"

url = polygon_url + "/vX/reference/tickers"
params = {
    "sort": "ticker",
    "active": "true",
    "search": search_term,
    "market": "crypto",
    "limit": 10,
    "order": "asc",
    "apiKey": polygon_api_key
}

r = requests.get(url, params=params)
if r.status_code == 200:
    tickers = r.json()["results"]
    print("tickers related to search query '%s':" % search_term)
    for t in tickers:
        print("  ticker:", t["ticker"])
        print("  name:", t["name"])
        print()
else:
    print("error: couldn't reach polygon api")

tickers related to search query 'dogecoin':
  ticker: X:DOGEUSD
  name: Dogecoin - United States Dollar



## set tickers to download

In [40]:
tickers = [
    "TSLA",
    "X:DOGEUSD"
]

## get closing prices for each ticker

In [62]:
day1 = "2020-05-01"
day2 = "2021-05-27"

data = dict()

for ticker in tickers:
    url = polygon_url + "/v2/aggs/ticker/" + ticker + "/range/1/day/"
    url += day1 + "/" + day2
    r = requests.get(url, params={"apiKey": polygon_api_key})
    if r.status_code == 200:
        times = list()
        prices = list()
        for entry in r.json()["results"]:
            times.append(entry["t"])
            prices.append(entry["c"])
        print(ticker + " prices from %s to %s:" % (day1, day2))
        print(prices)
        print()
        times_str = [str(datetime.datetime.fromtimestamp(t/1000)) for t in times]
        data[ticker] = {"times": times_str, "prices": prices, "n": len(prices)}
    else:
        print("error: couldn't reach polygon api for %s" % ticker)

TSLA prices from 2020-05-01 to 2021-05-27:
[140.264, 152.238, 153.642, 156.516, 156.008, 163.884, 162.258, 161.882, 158.192, 160.666, 159.834, 162.726, 161.602, 163.112, 165.52, 163.376, 163.774, 164.046, 161.162, 167, 179.62, 176.312, 176.592, 172.876, 177.132, 189.984, 188.134, 205.01, 194.568, 187.056, 198.18, 196.426, 198.358, 200.792, 200.18, 198.864, 200.356, 192.17, 197.196, 191.948, 201.87, 215.962, 223.926, 241.732, 274.316, 277.972, 273.176, 278.856, 308.93, 299.412, 303.36, 309.202, 300.128, 300.168, 328.6, 313.672, 318.466, 302.614, 283.4, 307.92, 295.298, 299.822, 297.498, 286.152, 297, 297.4, 297.004, 297.916, 290.542, 283.714, 274.878, 310.952, 324.2, 330.142, 367.128, 377.418, 375.706, 400.366, 409.996, 402.84, 404.668, 430.634, 447.75, 442.68, 498.32, 475.05, 447.37, 407, 418.32, 330.21, 366.28, 371.34, 372.72, 419.62, 449.76, 441.76, 423.43, 442.15, 449.39, 424.23, 380.36, 387.79, 407.34, 421.2, 419.07, 429.01, 448.16, 415.09, 425.68, 413.98, 425.3, 425.92, 434, 442.3

## save stock prices to file

In [64]:
import json

f = open("tsla_doge_prices.json", "w")
json.dump(data, f)
f.close()